# Predicting heart disease using Machine Learning

This notebook looks into using various Python based ML and data science libraries in an attempt to build a model which is capable of predicting wether or not someone has heart-disease or not based on their medical attributes

We're going to take following approach
1. Problem Definition
2. Data
3. Evaluation
4. Features
5. Modelling
6. Experimentation

## 1. Problem Definition

In a statement,
> Given clinical data for a person in the dataset, can we predict or not if they have heart disease?

## 2. Data

The original data came from Cleaveland data from the UCI Machine Learning Repository.
http://archive.ics.uci.edu/ml/datasets/heart+disease

There is also a version available on Kaggle. 
https://www.kaggle.com/ronitf/heart-disease-uci

## 3. Evaluation

> If we can reach 95% accuracy at predicting wether or not a patient has heart disease during the proof of concept, we'll puersue the project.

## 4. Features

**Create data dictionary**

* age - age in years
* sex - (1 = male; 0 = female)
* cp - chest pain type
 0: Typical angina: chest pain related decrease blood supply to the heart
 1: Atypical angina: chest pain not related to heart
 2: Non-anginal pain: typically esophageal spasms (non heart related)
 3: Asymptomatic: chest pain not showing signs of disease
* trestbps - resting blood pressure (in mm Hg on admission to the hospital) anything above 130-140 is typically cause for concern
* chol - serum cholestoral in mg/dl
* serum = LDL + HDL + .2 * triglycerides
* above 200 is cause for concern
* fbs - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
*'>126' mg/dL signals diabetes
* restecg - resting electrocardiographic results
 * 0: Nothing to note
 * 1: ST-T Wave abnormality
 * can range from mild symptoms to severe problems
 * signals non-normal heart beat
 * 2: Possible or definite left ventricular hypertrophy
 * Enlarged heart's main pumping chamber
* thalach - maximum heart rate achieved
* xang - exercise induced angina (1 = yes; 0 = no)
* oldpeak - ST depression induced by exercise relative to rest looks at stress of heart during excercise unhealthy heart will stress more
* slope - the slope of the peak exercise ST segment
 * 0: Upsloping: better heart rate with excercise (uncommon)
 * 1: Flatsloping: minimal change (typical healthy heart)
 * 2: Downslopins: signs of unhealthy heart
* ca - number of major vessels (0-3) colored by flourosopy
* colored vessel means the doctor can see the blood passing through
the more blood movement the better (no clots)
thal - thalium stress result
1,3: normal
6: fixed defect: used to be defect but ok now
7: reversable defect: no proper blood movement when excercising
target - have disease or not (1=yes, 0=no) (= the predicted attribute)

## Preparing the tools
We're going to use pandas, Matplotlib and NumPy for data analysis and manipulation.

In [ ]:
# Import all the tools we need

# Regular EDA (exploratory data analysis) and plotting libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# to plot graphs inline
%matplotlib inline

# models from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

#model evaluation
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve


## Load the data


In [ ]:
df=pd.read_csv("/kaggle/input/heart-disease-uci/heart.csv")
df.shape #(rows,columns)


## Data Exploration(exploratory Data Analysis, EDA)

The goal here is to find out more about the data and become subject matter expert on the dataset you're working with

1. What questions you are trying to solve?
2. what kind of data do we have and how do we treat diff types?
3. What's missing from the data and how do you deal with it?
4. Where are the outliers and why should you care about them?
5. How can you add, change or remove features to get more out of your data

In [ ]:
df["target"].value_counts().plot(kind="bar", color=["salmon","lightblue"])

In [ ]:
df.info()

In [ ]:
#Are there any missing values?
df.isna().sum()


In [ ]:
df.describe()

### Heart Disease Frequency according to sex


In [ ]:
df.sex.value_counts()

In [ ]:
#compare target columns with sex column
pd.crosstab(df.target, df.sex)

# based on our exisiting dataset 72 out of 96 have heart disease which is approx 75%
# in male 93 out of 207 have heart disease; almost 50%
Therefore on an avg target is 62.5% chances of having heart disease

In [ ]:
# Create a lot of crosstab

pd.crosstab(df.target, df.sex).plot(kind="bar",
                                   figsize=(10,6),
                                   color=["salmon","lightblue"])

plt.title("Heart Disease frequency per sex")
plt.xlabel("0 = No Disease, 1= Disease")
plt.ylabel("Amount")
plt.legend(["Female","Male"])
plt.xticks(rotation=0);

In [ ]:
df.head()

#Lest compare age, thalach(Max heart rate) and target



In [ ]:
plt.figure(figsize=(10,6))

#scatter with positive examples
plt.scatter(df.age[df.target==1],
           df.thalach[df.target==1],
           c="salmon");

#scatter with negative example

plt.scatter(df.age[df.target==0],
           df.thalach[df.target==0],
           c="black");

plt.title("Heart Disease in function of age and thalach")
plt.xlabel("Age")
plt.ylabel("Thalache (max heart rate)")
plt.legend(["Disease","No Disease"]);

In [ ]:
# check the dstribution of the age column with a histogram

df.age.plot.hist(figsize=(10,6));

## Lets check out chest pain type distribution
cp - chest pain type
* 0: Typical angina: chest pain related decrease blood supply to the heart
* 1: Atypical angina: chest pain not related to heart
* 2: Non-anginal pain: typically esophageal spasms (non heart related)
* 3: Asymptomatic: chest pain not showing signs of disease

In [ ]:
pd.crosstab( df.cp,df.target)

In [ ]:
#make thr cross tab visual

pd.crosstab(df.cp, df.target).plot(kind="bar", figsize=(10,6),
                                  color=["darkblue","lightblue"])

plt.title("Heart Disease frequency per chest pain type")
plt.xlabel("Chest Pain type")
plt.ylabel("Amount")
plt.legend(["No Disease","Disease"])
plt.xticks(rotation=0);

In [ ]:
# Make a corelation matrix

df.corr()

In [ ]:
# Lets make our corelation matrix a bit prettier

corr_matrix= df.corr()
fig, ax= plt.subplots(figsize=(10,6))
ax= sns.heatmap(corr_matrix,
               annot= True,
               linewidths=0.5,
               fmt=".2f",
            cmap="twilight_r");

## 5. Modelling

In [ ]:
#split data into X and y

X=df.drop("target", axis=1)
y=df["target"]

In [ ]:
#split data into training and test split
np.random.seed(42)

#Split into trin and test set

X_train, X_test, y_train,y_test= train_test_split(X,y,test_size=0.2)

In [ ]:
len(X_train), len(y_train)

### Now we've got our data in training and tests sets, its time to build a ML model using ML map and test, train and predict.
https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

We are going to try 3 diff ML models
1. Logistic Regression : https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
2. K-nearest neighbour classifier
3. RandomForrestClassifier

In [ ]:
# Put models in a dictionary

models={"Logistic Regression": LogisticRegression(),
       "KNN":KNeighborsClassifier(),
       "Random Forest": RandomForestClassifier()}

#create a function to fit and score models
def fit_and_score(models,X_train, X_test, y_train, y_test):
    """
    Fits and evaluates given ML models.
    models: a dict of diff SciKit learn ML models
    X_test :  test set (no label)
    X_train: training set (no label)
    y_train : training labels
    y_test : test labels
    """
    
    #set random seed
    np.random.seed(42)
    #Make dictionary to keep model scores
    
    model_scores={}
    
    #Loop through models
    for name, model in models.items():
        #fit model to data
        model.fit(X_train, y_train)
        #evaluate the model and store in score dict
        model_scores[name]= model.score(X_test, y_test)
    return model_scores

In [ ]:
model_scores= fit_and_score(models=models,
                           X_train=X_train,
                           X_test=X_test,
                           y_train=y_train,
                           y_test=y_test)

model_scores

### Model comparison

In [ ]:
model_compare =pd.DataFrame(model_scores, index=["accuracy"])
model_compare.T.plot.bar();

Now we have got a baseline model and we know a model's first prediction is not something we must count on.
Now we've got a baseline model... and we know a model's first predictions aren't always what we should based our next steps off. What should we do?

Let's look at the following:

1. Hyperparameter tuning
2. Feature importance
3. Confusion matrix
4. Cross-validation
5. Precision
6. Recall
7. F1 score
8. Classification report
9. ROC curve
10. Area under the curve (AUC)

## Hyper Parameter tuning (By hand)

In [ ]:
#lets tune KNN
train_scores = []
test_scores =  []

#Create a list of diff values of n_neighbours
neighbors = range(1,21)

#Setup KNN Instance
knn = KNeighborsClassifier()

#Loop thriugh diff n_neighbours
for i in neighbors:
    knn.set_params(n_neighbors = i)
    
    #Fit the algo
    knn.fit(X_train, y_train)
    
    #Update the training scores list
    train_scores.append(knn.score(X_train, y_train))
    
    #Update the test scores
    test_scores.append(knn.score(X_test, y_test))
    

In [ ]:
train_scores

In [ ]:
test_scores

In [ ]:
#Visualize

plt.plot(neighbors, train_scores, label="Train score")
plt.plot(neighbors, test_scores, label="Test score")
plt.xticks(np.arange(1,21,1))
plt.xlabel("Number of neighbors")
plt.ylabel("Model score")
plt.legend()

print(f"Maximum KNN score on the test data :{max(test_scores)*100:.2f}%")

### We have achieved an improvement in our KNN score as compared to 68% previously, now with hyperparameter tuning we 
### have got upto 75% when we use 11 neighbors.
Although it is an improvement but it is not close to the the other two models which had scores of 
{'Logistic Regression': 0.8852459016393442,
 'KNN': 0.6885245901639344,
 'Random Forest': 0.8360655737704918}

Thus we can focus on other models and try tuning it to achive our goal which is 95% accuracy

## Hyperparameter tuning by RandomizedSearchCV

We are going to tune 
* LogisticRegresion model and 
* RandomForestClassifier 

using RandomizedSearchCV

In [ ]:
#  Create a hyperparameter grid for LogisticRegression
log_reg_grid = {"C" : np.logspace(-4, 4, 20),
            "solver": ["liblinear"]}

#Create a hyperparam grid for RandomForestClassifier
rf_grid = {"n_estimators" : np.arange(10, 1000, 50),
         "max_depth" : [None,3,5,10],
         "min_samples_split" : np.arange(2, 20, 2),
         "min_samples_leaf": np.arange(1, 20, 2)}

Now grid is setup for models, lets us tune them using RandomizedSearchCV


In [ ]:
# Tune LogisticRegression

np.random.seed(42)

#Setup random hyperparams search for LogisticRegression

rs_log_reg= RandomizedSearchCV(LogisticRegression(),
                              param_distributions = log_reg_grid,
                              cv=5,
                              n_iter=20,
                              verbose=True)

#Fit random hyperparam search model for LogisticRegression
rs_log_reg.fit(X_train, y_train)

In [ ]:
rs_log_reg.best_params_

In [ ]:
rs_log_reg.score(X_test, y_test)

In [ ]:
# Now we have tuned for LogisticRegression
# Tune RandomFOrest

In [ ]:
np.random.seed(42)

#Setup random hyperparam search for RandomFOrestClassifier
rs_ref = RandomizedSearchCV(RandomForestClassifier(),
                          param_distributions = rf_grid,
                          cv=5,
                          n_iter= 20,
                          verbose=True)
# Fit random hyperparam search model for RandomForestCLassifier

rs_ref.fit(X_train, y_train)



In [ ]:
#Finding best params
rs_ref.best_params_

In [ ]:
#Evaluate RandomSearchCV search on RandomForestClassifier model
rs_ref.score(X_test, y_test)

{'Logistic Regression': 0.8852459016393442, 'KNN': 0.6885245901639344, 'Random Forest': 0.8360655737704918}

This tuning has improved our score from 83.60 to 86.88 but our LogisticRegression has more accuracy

# Lets use GridSearchCV to exhaustively over specifed params
Since our LogisticRegression model offers the best accuracy scores so far, so we'll try to improve using GridSeacrhCV

In [ ]:
# Different hyperparameters for LR Model
log_reg_grid = {"C": np.logspace(-4,4,30),
               "solver": ["liblinear"]}

#Setup grid hyperparameter search for LogisticRegression
gs_log_reg = GridSearchCV(LogisticRegression(),
                         param_grid= log_reg_grid,
                         cv=5,
                         verbose=True)
#Fit grid hyperparam search model

gs_log_reg.fit(X_train, y_train);

In [ ]:
gs_log_reg.best_params_

In [ ]:
# Evaluate GridSearchCV for LR model
gs_log_reg.score(X_test, y_test)

In [ ]:
model_scores

## Evaluate our tuned ML classifier, beyond accuracy
* ROC Curve
* AUC Score
* Confusion matrix
* CLassification score
* Precision
* Recall
* F1 score
... and it would be great if CV was used wherever possible
to make comparison and evaluate our trained model, we need to make predictions....

In [ ]:
# make predictions
y_preds=gs_log_reg.predict(X_test)

In [ ]:
y_preds

In [ ]:
# Import ROC curve fucntion but we have done this previously.
#  roc curve and calculate AUC metric
plot_roc_curve(gs_log_reg, X_test, y_test)

In [ ]:
# Confusion matrix
sns

In [ ]:
sns.set(font_scale=1.5)

def plot_conf_mat(y__test, y_preds):
    """
    Plots a confusion matrix using Seaborn's heatmap().
    """
    fig, ax = plt.subplots(figsize=(3, 3))
    ax = sns.heatmap(confusion_matrix(y_test, y_preds),
                     annot=True, # Annotate the boxes
                     cbar=False)
    plt.xlabel("Predicted label") # predictions go on the x-axis
    plt.ylabel("True label") # true labels go on the y-axis 
    
plot_conf_mat(y_test, y_preds)

# Now we have got out ROC curve and AUC matrics and confusion matric, Lets get Classification report
and cross validated precision, recall and F1 score

In [ ]:
print(classification_report(y_test, y_preds))

### Calculate evaluation matrics using CV

precision, recall and F1 score of our model using cross_val_score

In [ ]:
# check our best hyperparams
gs_log_reg.best_params_

In [ ]:
# create a new classifier with best params

clf= LogisticRegression(C=0.20433597178569418,
                       solver="liblinear")
# Cross validated accuracy
cv_acc= cross_val_score(clf,
                       X,
                       y,
                       cv=5,
                       scoring="accuracy")
cv_acc

In [ ]:
cv_acc=np.mean(cv_acc)
cv_acc

In [ ]:
# Cross validated precision
cv_precision= cross_val_score(clf,
                       X,
                       y,
                       cv=5,
                       scoring="precision")

cv_precision=np.mean(cv_precision)
cv_precision

In [ ]:
# Cross validated recall
cv_recall= cross_val_score(clf,
                       X,
                       y,
                       cv=5,
                       scoring="recall")

cv_recall=np.mean(cv_recall)
cv_recall

In [ ]:
# Cross validated f1
cv_f1= cross_val_score(clf,
                       X,
                       y,
                       cv=5,
                       scoring="f1")

cv_f1=np.mean(cv_f1)
cv_f1

In [ ]:
# putting it in a graph visualize 
cv_metrics= pd.DataFrame({"Accuracy": cv_acc,
                         "Precision": cv_precision,
                         "Recall": cv_recall,
                         "f1": cv_f1},
                        index=[0])

cv_metrics.T.plot.bar(title="Cross validated classification metrics", legend=False)


## Feature importance

It is another way of asking which features contribute most to the outcome of the model and how did they contribute?

Finding fearure importance is diff for each ML model. oneway to find features which are important is by googling "Model name" feature importance.

Lets find the features important or our LogisticRegression model

In [ ]:
gs_log_reg.best_params_

In [ ]:
clf= LogisticRegression(C=0.20433597178569418,
                       solver="liblinear")

clf.fit(X_train, y_train);

In [ ]:
#check coef attribute to give how each parameter contributes to our target labels
clf.coef_

In [ ]:
# Match the coef's of features to columns

feature_dict = dict(zip(df.columns, list(clf.coef_[0])))
feature_dict


In [ ]:
# Visualize feature importance

feature_df = pd.DataFrame(feature_dict, index=[0])
feature_df.T.plot.bar(title="Feature importance", legend=False);

In [ ]:
# if the value is -ve negative corelation

pd.crosstab(df.sex, df.target)

In [ ]:
pd.crosstab(df.slope, df.target)

slope - the slope of the peak exercise ST segment
* 0: Upsloping: better heart rate with excercise (uncommon)
* 1: Flatsloping: minimal change (typical healthy heart)
* 2: Downslopins: signs of unhealthy heart

## 6. Experimentation

If you have not hit your evaluation metric yet, ask yourself:
* Could you collect more data/samples?
* Could try a better model? Like Catboost or XGBoost?
* Could you improve the current models?( beyond what we have done so far)
* If your model is good enough (you have hit your evaluation metric) how would you export it and share with others?
* Ask more questions, reachout, ask on stackoverflow